
# Pandas — Guia Completo e Didático (com dados sintéticos) — **v2 (com visualizações avançadas)**

**Objetivo:** fornecer um *notebook* auto-contido e pronto para aula/estudo, cobrindo **leitura**, **inspeção**, **limpeza**, **seleção**, **datas**, **agregação**, **reshape** (*pivot/melt*), **merge/join/concat**, **ranking/janelas**, **tipos/categorias**, **I/O**, **performance**, **visualização (matplotlib)**, **exercícios/gabaritos** e um **anexo** sobre **módulo × pacote**.  
> **NOVIDADE**: Seção **16** com **Matplotlib, Seaborn, Plotly, Altair e Bokeh**.

## Sumário
1. Ambiente e versões  
2. Geração de dados sintéticos  
3. Leitura de dados: CSV/Excel/JSON  
4. Inspeção, seleção e `query`  
5. Limpeza e preparação  
6. Datas e séries temporais  
7. Agregações e `groupby` avançado  
8. Reshape: `pivot`, `pivot_table`, `melt`  
9. Junções: `merge`, `join`, `concat`  
10. Ordenação, ranking e janelas  
11. Tipos e categorias  
12. Entrada/Saída  
13. Performance  
14. Visualização (matplotlib)  
15. Exercícios + Gabaritos  
16. **Visualizações avançadas (Matplotlib, Seaborn, Plotly, Altair, Bokeh)**  
17. Anexo: Módulo × Pacote em Python e `__init__.py`


In [ ]:

import sys, platform
import pandas as pd
import numpy as np

print("Python:", sys.version.split()[0])
print("Pandas:", pd.__version__)
print("Numpy:", np.__version__)
print("OS:", platform.platform())



## 1) Geração de dados sintéticos (auto-contido)

Criamos:
- **Vendas** (`df_vendas`): data, região, produto, **categoria**, **id_cliente**, **nome_cliente**, **valor**
- **Clientes (dim)** (`df_clientes`): id, nome, região, renda
- **Filmes (JSON)**: lista de objetos `{titulo, ano, genero, nota}`
- **Planilha Excel** com duas abas: `Compras` e `Clientes`

Também salvamos versões temporárias (CSV/JSON/XLSX) em `data/`.


In [ ]:

from pathlib import Path
from datetime import datetime, timedelta
import json
import pandas as pd
import numpy as np

np.random.seed(42)

base = Path("data")
base.mkdir(exist_ok=True)

# --- Clientes (dimensão) ---
n_clientes = 120
regioes = ["Norte", "Nordeste", "Centro-Oeste", "Sudeste", "Sul"]
nomes = [f"Cliente {i:03d}" for i in range(1, n_clientes+1)]
ids = np.arange(1, n_clientes+1)

df_clientes = pd.DataFrame({
    "id_cliente": ids,
    "nome_cliente": nomes,
    "regiao": np.random.choice(regioes, size=n_clientes, replace=True),
    "renda_mensal": np.random.gamma(2.0, 1500, size=n_clientes).round(2)
})

# --- Vendas ---
dias = pd.date_range("2024-01-01", periods=365, freq="D")
produtos = [f"Produto_{i:02d}" for i in range(1, 15)]
categorias = {
    "Bebidas": produtos[0:4],
    "Alimentos": produtos[4:8],
    "Higiene": produtos[8:11],
    "Limpeza": produtos[11:14]
}
cat_por_prod = {p: c for c, plist in categorias.items() for p in plist}

linhas = 15000
df_vendas = pd.DataFrame({
    "data": np.random.choice(dias, size=linhas, replace=True),
    "id_cliente": np.random.choice(ids, size=linhas, replace=True),
    "produto": np.random.choice(produtos, size=linhas, replace=True),
    "valor": np.abs(np.random.normal(120, 60, size=linhas)).clip(5, None).round(2)
})
df_vendas["categoria"] = df_vendas["produto"].map(cat_por_prod)
df_vendas = df_vendas.merge(df_clientes, on="id_cliente", how="left")
df_vendas = df_vendas[[
    "data", "regiao", "id_cliente", "nome_cliente", "produto", "categoria", "valor", "renda_mensal"
]].sort_values("data", ignore_index=True)

# --- JSON (filmes) ---
generos = ["Ação", "Drama", "Comédia", "Ficção", "Documentário"]
filmes = [{
    "titulo": f"Filme {i:02d}",
    "ano": int(np.random.choice(np.arange(1980, 2025))),
    "genero": np.random.choice(generos),
    "nota": round(np.random.uniform(1, 10), 1)
} for i in range(1, 51)]

# --- Salva CSV/JSON/Excel ---
df_vendas.to_csv(base / "vendas.csv", index=False, encoding="utf-8")
df_clientes.to_csv(base / "clientes.csv", index=False, encoding="utf-8")
with open(base / "filmes.json", "w", encoding="utf-8") as f:
    json.dump(filmes, f, ensure_ascii=False, indent=2)

with pd.ExcelWriter(base / "dados.xlsx") as w:
    df_vendas.to_excel(w, sheet_name="Compras", index=False)
    df_clientes.to_excel(w, sheet_name="Clientes", index=False)

len(df_vendas), len(df_clientes), len(filmes)



## 2) Leitura de dados (CSV, Excel, JSON)


In [ ]:

df_csv = pd.read_csv("data/vendas.csv", parse_dates=["data"])
df_xlsx_compras = pd.read_excel("data/dados.xlsx", sheet_name="Compras", parse_dates=["data"])
df_xlsx_clientes = pd.read_excel("data/dados.xlsx", sheet_name="Clientes")
df_json = pd.read_json("data/filmes.json", orient="records")

print(df_csv.shape, df_xlsx_compras.shape, df_xlsx_clientes.shape, df_json.shape)
df_csv.head(3)


In [ ]:

display(df_csv.info())
display(df_csv.describe(numeric_only=True))



## 3) Inspeção, seleção e `query`


In [ ]:

df = df_csv.copy()

primeiras_linhas = df.head(5)
colunas_escolhidas = df[["data", "regiao", "valor"]].head(3)
por_posicao = df.iloc[0:3, 0:4]

sudeste_maior_200 = df[(df["regiao"] == "Sudeste") & (df["valor"] > 200)].head(5)

q1 = df.query("regiao == 'Sul' and valor < 50").head(3)

df_espaco = df.rename(columns={"nome_cliente": "Nome do Cliente"})
q2 = df_espaco.query("`Nome do Cliente`.str.contains('010')", engine="python").head(3)

primeiras_linhas, colunas_escolhidas, por_posicao, sudeste_maior_200, q1, q2



## 4) Limpeza e preparação


In [ ]:

df = df_csv.copy()

mask = np.random.rand(len(df)) < 0.01
df.loc[mask, "valor"] = np.nan
mask2 = np.random.rand(len(df)) < 0.01
df.loc[mask2, "categoria"] = np.nan

percent_nulos = df.isna().mean().round(4) * 100
df["valor"] = df["valor"].fillna(df["valor"].median())
df["categoria"] = df["categoria"].fillna("Desconhecida")

antes = len(df)
df_dup = pd.concat([df, df.iloc[0:2]], axis=0, ignore_index=True)
apos = len(df_dup)
df_sem_dup = df_dup.drop_duplicates()

percent_nulos, (antes, apos, len(df_sem_dup))



## 5) Datas e séries temporais


In [ ]:

df = df_csv.copy()
df["data"] = pd.to_datetime(df["data"])

mensal_regiao = (df
    .groupby([pd.Grouper(key="data", freq="MS"), "regiao"])["valor"]
    .sum()
    .unstack("regiao", fill_value=0)
)

resample_mensal = (df.set_index("data")["valor"]
    .resample("MS").sum()
)

mensal_regiao.head(3), resample_mensal.head(3)



## 6) Agregações e `groupby` avançado


In [ ]:

df = df_csv.copy()

agg_cat = (df.groupby("categoria")["valor"]
             .agg(qtd="count", soma="sum", media="mean", mediana="median")
             .sort_values("soma", ascending=False))

df["z_valor_regiao"] = (df.groupby("regiao")["valor"]
                          .transform(lambda s: (s - s.mean())/s.std(ddof=0)))

df["mes"] = df["data"].astype("datetime64[M]")
somas = (df.groupby(["mes","regiao","produto"])["valor"]
           .sum()
           .reset_index())
somas["rank"] = (somas.groupby(["mes","regiao"])["valor"]
                       .rank(ascending=False, method="first"))
top3 = somas.query("rank <= 3").sort_values(["mes","regiao","valor"], ascending=[True, True, False])

agg_cat.head(3), df[["regiao","valor","z_valor_regiao"]].head(3), top3.head(9)



## 7) Reshape de dados


In [ ]:

df = df_csv.copy()
df["mes"] = df["data"].astype("datetime64[M]")

tabela = (df.pivot_table(values="valor", index="mes", columns="regiao",
                         aggfunc="sum", fill_value=0))

tabela_reset = tabela.reset_index()
tabela_melt = tabela_reset.melt(id_vars="mes", var_name="regiao", value_name="valor")

media_prod_cat = (df.groupby(["produto","categoria"])["valor"]
                    .mean().reset_index())

pivot_media = media_prod_cat.pivot(index="produto", columns="categoria", values="valor").fillna(0)

tabela.head(3), tabela_melt.head(3), pivot_media.head(3)



## 8) Junções e concatenação


In [ ]:

df_v = df_csv.copy()
df_c = df_xlsx_clientes.copy()

enriquecido = df_v.merge(df_c[["id_cliente","renda_mensal"]], on="id_cliente", how="left")

left = (df_v.groupby("regiao")["valor"].sum().to_frame("soma_valor"))
right = (df_v.groupby("regiao")["valor"].mean().to_frame("media_valor"))
por_indice = left.join(right, how="inner")

part1 = df_v.iloc[:100].reset_index(drop=True)
part2 = df_v.iloc[100:200].reset_index(drop=True)
empilhado = pd.concat([part1, part2], axis=0, ignore_index=True)

lado_a_lado = pd.concat([left, right], axis=1)

enriquecido.head(3), por_indice, empilhado.shape, lado_a_lado.head(3)



## 9) Ordenação, ranking e janelas (`rolling`, `expanding`)


In [ ]:

df = df_csv.copy().sort_values(["data","valor"])

serie_diaria = df.groupby("data")["valor"].sum().sort_index()
mm7 = serie_diaria.rolling(window=7, min_periods=1).mean()
acum = serie_diaria.expanding().sum()

serie_diaria.head(3), mm7.head(10), acum.head(5)



## 10) Tipos e categorias


In [ ]:

from pandas.api.types import CategoricalDtype

df = df_csv.copy()
ordem = ["Norte", "Nordeste", "Centro-Oeste", "Sudeste", "Sul"]
cat_tipo = CategoricalDtype(categories=ordem, ordered=True)
df["regiao"] = df["regiao"].astype(cat_tipo)

df["regiao"].dtype, df["regiao"].cat.categories, df["regiao"].cat.ordered



## 11) Entrada/Saída


In [ ]:

out = Path("output")
out.mkdir(exist_ok=True)

amostra = df_csv.sample(1000, random_state=0).sort_values("data")

amostra.to_csv(out / "amostra.csv", index=False, encoding="utf-8")
amostra.to_excel(out / "amostra.xlsx", index=False)
amostra.to_json(out / "amostra.json", orient="records", force_ascii=False, indent=2)

parquet_info = "Parquet não gerado (sem engine)."
try:
    amostra.to_parquet(out / "amostra.parquet", index=False)
    parquet_info = "Parquet gerado com sucesso."
except Exception as e:
    parquet_info = f"Falha no Parquet: {type(e).__name__}: {e}"

parquet_info, list(out.iterdir())[:5]



## 12) Performance: `dtype`, `usecols`, `chunksize`


In [ ]:

tipos = {
    "regiao": "category",
    "id_cliente": "int32",
    "nome_cliente": "string",
    "produto": "string",
    "categoria": "category",
    "valor": "float32",
    "renda_mensal": "float32",
}
subset = pd.read_csv("data/vendas.csv",
                     usecols=["data","regiao","id_cliente","produto","valor"],
                     dtype={"regiao":"category","id_cliente":"int32","produto":"string","valor":"float32"},
                     parse_dates=["data"])
mem_mb = subset.memory_usage(deep=True).sum() / (1024**2)

somas = {}
for chunk in pd.read_csv("data/vendas.csv", chunksize=5000):
    g = chunk.groupby("produto")["valor"].sum()
    for k,v in g.items():
        somas[k] = somas.get(k, 0.0) + v
somas_res = pd.Series(somas).sort_values(ascending=False).head(5)

mem_mb, somas_res



## 13) Visualização (matplotlib)


In [ ]:

import matplotlib.pyplot as plt

df = df_csv.copy()
serie_diaria = df.groupby("data")["valor"].sum().sort_index()

plt.figure(figsize=(9,4))
plt.plot(serie_diaria.index, serie_diaria.values)
plt.title("Total diário de vendas")
plt.xlabel("Data"); plt.ylabel("Valor")
plt.tight_layout(); plt.show()


In [ ]:

df = df_csv.copy()
df["mes"] = df["data"].astype("datetime64[M]")
mensal = (df.groupby(["mes","regiao"])["valor"]
            .sum()
            .unstack("regiao", fill_value=0))

plt.figure(figsize=(10,4))
mensal.plot(kind="bar", ax=plt.gca())
plt.title("Total mensal por região")
plt.xlabel("Mês"); plt.ylabel("Valor")
plt.tight_layout(); plt.show()


In [ ]:

plt.figure(figsize=(8,4))
plt.hist(df_csv["valor"], bins=40)
plt.title("Distribuição dos valores de venda")
plt.xlabel("Valor"); plt.ylabel("Frequência")
plt.tight_layout(); plt.show()


In [ ]:

ordem = ["Bebidas","Alimentos","Higiene","Limpeza"]
df_ord = df_csv[df_csv["categoria"].isin(ordem)].copy()
df_ord["categoria"] = pd.Categorical(df_ord["categoria"], categories=ordem, ordered=True)

plt.figure(figsize=(8,4))
df_ord.boxplot(column="valor", by="categoria")
plt.title("Boxplot de valor por categoria"); plt.suptitle("")
plt.xlabel("Categoria"); plt.ylabel("Valor")
plt.tight_layout(); plt.show()


In [ ]:

g = (df_csv.groupby(["id_cliente","nome_cliente"])["valor"]
       .mean()
       .rename("valor_medio")
       .reset_index()
       .merge(df_clientes[["id_cliente","renda_mensal"]], on="id_cliente", how="left"))

plt.figure(figsize=(8,4))
plt.scatter(g["renda_mensal"], g["valor_medio"], s=10)
plt.title("Renda mensal x Valor médio por cliente")
plt.xlabel("Renda mensal"); plt.ylabel("Valor médio de compra")
plt.tight_layout(); plt.show()



## 14) Exercícios (com gabarito)


In [ ]:

# E1
df = df_csv.copy()
e1 = (df.groupby(["regiao","produto"])["valor"].sum()
        .reset_index()
        .sort_values(["regiao","valor"], ascending=[True, False])
        .groupby("regiao").head(5))
e1.head(10)


In [ ]:

# E2 - top3 por mês/região
df = df_csv.copy()
df["mes"] = df["data"].astype("datetime64[M]")
somas = (df.groupby(["mes","regiao","produto"])["valor"]
           .sum()
           .reset_index())
somas["rank"] = somas.groupby(["mes","regiao"])["valor"].rank(ascending=False, method="first")
e2 = somas.query("rank <= 3").sort_values(["mes","regiao","valor"], ascending=[True, True, False])
e2.head(12)


In [ ]:

# E3 - clientes que compraram em mais de uma categoria
g = (df_csv.groupby(["id_cliente","nome_cliente"])["categoria"]
        .nunique()
        .rename("qtd_categorias"))
e3 = g[g > 1].reset_index().sort_values("qtd_categorias", ascending=False)
e3.head(10)


In [ ]:

# E4 - tabela mensal e salvar
out = Path("output")
out.mkdir(exist_ok=True)
tabela_mensal = (df_csv
                 .assign(mes=df_csv["data"].astype("datetime64[M]"))
                 .groupby(["mes","regiao"])["valor"].sum()
                 .unstack("regiao", fill_value=0))
tabela_mensal.to_excel(out / "tabela_mensal.xlsx")
tabela_mensal.head(3)


In [ ]:

# E5 - média móvel de 7 dias
import matplotlib.pyplot as plt
serie_diaria = (df_csv.groupby("data")["valor"]
                .sum()
                .sort_index())
mm7 = serie_diaria.rolling(window=7, min_periods=1).mean()

plt.figure(figsize=(9,4))
plt.plot(serie_diaria.index, serie_diaria.values, label="Diário")
plt.plot(mm7.index, mm7.values, label="MM7")
plt.title("Total diário e média móvel de 7 dias")
plt.xlabel("Data"); plt.ylabel("Valor")
plt.legend()
plt.tight_layout(); plt.show()



## 16) Visualizações avançadas por biblioteca (Matplotlib, Seaborn, Plotly, Altair, Bokeh)

> As células abaixo utilizam `df_vendas` e `df_clientes` já carregados.  
> **Dica**: execute a célula de preparação antes de plotar.


In [ ]:

# Preparação de agregações para os gráficos avançados
import pandas as pd
import numpy as np

df_vendas = df_vendas.copy()
df_vendas["data"] = pd.to_datetime(df_vendas["data"])
df_vendas["mes"] = df_vendas["data"].values.astype("datetime64[M]")

serie_diaria = df_vendas.groupby("data")["valor"].sum().sort_index()
mensal_regiao = (df_vendas.groupby(["mes","regiao"])["valor"]
                 .sum().unstack("regiao", fill_value=0).sort_index())
por_categoria = df_vendas.groupby("categoria")["valor"].sum().sort_values(ascending=False)
top_produtos = (df_vendas.groupby("produto")["valor"].sum()
                .sort_values(ascending=False).head(10).reset_index())
client_media = (df_vendas.groupby(["id_cliente","nome_cliente"])["valor"].mean()
                .rename("valor_medio").reset_index()
                .merge(df_clientes[["id_cliente","renda_mensal","regiao"]], on="id_cliente", how="left"))



### 16.1 Matplotlib — base sólida, anotações e layout


In [ ]:

import matplotlib.pyplot as plt
import matplotlib.dates as mdates

fig = plt.figure(figsize=(11, 4.5))
ax = plt.gca()

ax.plot(serie_diaria.index, serie_diaria.values, linewidth=2)
ax.set_title("Vendas diárias (R$)", pad=12)
ax.set_xlabel("Data"); ax.set_ylabel("Valor")
ax.grid(True, alpha=.3)

ax.xaxis.set_major_locator(mdates.MonthLocator(interval=1))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b/%Y'))
fig.autofmt_xdate(rotation=0)

i_max = serie_diaria.idxmax()
ax.annotate(f"Pico\\nR$ {serie_diaria.max():,.0f}".replace(',', 'X').replace('.', ',').replace('X','.'),
            xy=(i_max, serie_diaria.max()), xytext=(15, 30),
            textcoords="offset points", arrowprops=dict(arrowstyle="->"))

plt.tight_layout(); plt.show()


In [ ]:

# Barras agrupadas (mensal × região)
fig = plt.figure(figsize=(11,4.5))
ax = plt.gca()
mensal_regiao.plot(kind="bar", ax=ax)
ax.set_title("Vendas mensais por região (R$)")
ax.set_xlabel("Mês"); ax.set_ylabel("Valor")
ax.grid(axis="y", alpha=.3)
plt.tight_layout(); plt.show()


In [ ]:

# Histograma + média/mediana
fig = plt.figure(figsize=(9,4))
ax = plt.gca()
ax.hist(df_vendas["valor"], bins=50)
ax.set_title("Distribuição de valores de venda")
ax.set_xlabel("Valor da venda"); ax.set_ylabel("Frequência")
ax.axvline(df_vendas["valor"].mean(), linestyle="--", linewidth=1)
ax.axvline(df_vendas["valor"].median(), linestyle=":", linewidth=1)
ax.grid(True, alpha=.25)
plt.tight_layout(); plt.show()


In [ ]:

# Boxplot por categoria (ordenado)
ordem = por_categoria.index.tolist()
df_box = df_vendas[df_vendas["categoria"].isin(ordem)].copy()
df_box["categoria"] = pd.Categorical(df_box["categoria"], categories=ordem, ordered=True)

fig = plt.figure(figsize=(9,4))
ax = plt.gca()
df_box.boxplot(column="valor", by="categoria", ax=ax)
ax.set_title("Boxplot: valor por categoria"); ax.set_xlabel("Categoria"); ax.set_ylabel("Valor")
plt.suptitle("")
plt.tight_layout(); plt.show()



### 16.2 Seaborn — estatística visual rápida


In [ ]:

import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme(context="talk", style="whitegrid")

tmp = df_vendas.groupby(["data","regiao"])["valor"].sum().reset_index()

plt.figure(figsize=(11,4.5))
sns.lineplot(data=tmp, x="data", y="valor", hue="regiao", estimator=None)
plt.title("Série diária por região (soma)")
plt.xlabel("Data"); plt.ylabel("Valor"); plt.tight_layout(); plt.show()


In [ ]:

mensal_long = mensal_regiao.reset_index().melt(id_vars="mes", var_name="regiao", value_name="valor")
plt.figure(figsize=(10,4.5))
sns.barplot(data=mensal_long, x="mes", y="valor", hue="regiao")
plt.title("Mensal por região (soma)")
plt.xlabel("Mês"); plt.ylabel("Valor"); plt.xticks(rotation=0)
plt.tight_layout(); plt.show()


In [ ]:

plt.figure(figsize=(10,4.5))
sns.boxenplot(data=df_box, x="categoria", y="valor")
sns.stripplot(data=df_box.sample(min(3000, len(df_box)), random_state=0), x="categoria", y="valor", size=2, alpha=.3)
plt.title("Distribuição por categoria (boxen + pontos)")
plt.xlabel("Categoria"); plt.ylabel("Valor")
plt.tight_layout(); plt.show()


In [ ]:

num = df_clientes[["renda_mensal"]].copy()
corr = num.corr()
plt.figure(figsize=(4,3.5))
sns.heatmap(corr, annot=True, fmt=".2f", square=True)
plt.title("Correlação (clientes)"); plt.tightLayout = plt.tight_layout; plt.tight_layout(); plt.show()


In [ ]:

g = sns.jointplot(data=client_media, x="renda_mensal", y="valor_medio", kind="hex", height=5)
g.fig.suptitle("Renda × Valor médio de compra (hexbin)", y=1.02)
plt.show()



### 16.3 Plotly — interatividade (zoom, hover, range slider)


In [ ]:

import plotly.express as px
import plotly.graph_objects as go


In [ ]:

tmp = df_vendas.groupby("data")["valor"].sum().reset_index()
fig = px.line(tmp, x="data", y="valor", title="Vendas diárias (interativo)")
fig.update_xaxes(rangeslider_visible=True)
fig.show()


In [ ]:

mensal_long_sorted = (mensal_regiao
                      .reset_index()
                      .melt(id_vars="mes", var_name="regiao", value_name="valor")
                      .sort_values(["mes","regiao"]))
fig = px.bar(mensal_long_sorted, x="mes", y="valor", color="regiao", barmode="group",
             title="Mensal por região (interativo)")
fig.update_layout(xaxis_tickformat="%b\n%Y")
fig.show()


In [ ]:

fig = px.scatter(client_media, x="renda_mensal", y="valor_medio", color="regiao",
                 hover_data=["nome_cliente"], facet_col="regiao", facet_col_wrap=3,
                 title="Renda × Valor médio por cliente (facet por região)")
fig.update_traces(marker=dict(size=6, opacity=.7))
fig.show()


In [ ]:

treemap_df = (df_vendas.groupby(["categoria","produto"])["valor"].sum()
              .reset_index())
fig = px.treemap(treemap_df, path=["categoria","produto"], values="valor",
                 title="Participação por categoria/produto")
fig.show()



### 16.4 Altair — declarações e interações elegantes


In [ ]:

import altair as alt
alt.data_transformers.disable_max_rows()


In [ ]:

tmp = df_vendas.groupby(["data","regiao"])["valor"].sum().reset_index()
select_reg = alt.selection_point(fields=["regiao"], bind="legend")

chart = (alt.Chart(tmp)
         .mark_line()
         .encode(
             x="data:T",
             y=alt.Y("valor:Q", title="Valor"),
             color="regiao:N",
             tooltip=["data:T","regiao:N","valor:Q"]
         )
         .add_params(select_reg)
         .transform_filter(select_reg)
         .properties(width=700, height=350, title="Série diária por região (Altair)"))
chart


In [ ]:

mensal_long = (mensal_regiao.reset_index()
               .melt(id_vars="mes", var_name="regiao", value_name="valor"))

brush = alt.selection_interval(encodings=["x"])

base = (alt.Chart(mensal_long)
        .mark_bar()
        .encode(x="mes:T", y="valor:Q", color="regiao:N", tooltip=["mes:T","regiao:N","valor:Q"])
        .properties(width=700, height=180))

upper = base.add_params(brush)
lower = base.transform_filter(brush)

upper & lower


In [ ]:

heat = (alt.Chart(mensal_long)
        .mark_rect()
        .encode(
            x=alt.X("mes:T", title="Mês"),
            y=alt.Y("regiao:N", title="Região"),
            tooltip=["mes:T","regiao:N","valor:Q"],
            fill="valor:Q"
        )
        .properties(width=700, height=220, title="Heatmap: mensal × região"))
heat



### 16.5 Bokeh — interativo com hover, zoom e pan


In [ ]:

from bokeh.plotting import figure, show
from bokeh.models import HoverTool, ColumnDataSource
from bokeh.io import output_notebook

output_notebook()

tmp = df_vendas.groupby("data")["valor"].sum().reset_index()
source = ColumnDataSource(tmp.rename(columns={"data":"x","valor":"y"}))

p = figure(x_axis_type="datetime", width=900, height=320, title="Vendas diárias (Bokeh)")
p.line("x","y", source=source, line_width=2)
p.add_tools(HoverTool(tooltips=[("Data","@x{%F}"), ("Valor","@y{0,0.00}")], formatters={"@x":"datetime"}))
p.toolbar_location = "right"
p.xaxis.axis_label = "Data"; p.yaxis.axis_label = "Valor"

show(p)



### Dicas finais de uso profissional
- Pré-agregue no Pandas antes de plotar, para manter interatividade fluida.
- Formate eixos (datas/unidades) e padronize títulos/labels.
- Anote picos/vales e metas quando útil (Matplotlib/Plotly).
- **Storytelling**: overview → cortes por dimensão → top‑N → drilldown.



## 17) **Anexo** — Python: Módulo × Pacote e `__init__.py`



- **Módulo**: **um arquivo** `.py` (ex.: `modulo.py`).
- **Pacote**: **uma pasta** com um `__init__.py` (executado ao importar o pacote).
- O método **`__init__` de classe** não é “construtor” em Python; quem cria a instância é `__new__`. O `__init__` apenas **inicializa**.

```
meu_projeto/
└── pacote_exemplo/
    ├── __init__.py
    └── modulo.py
```

**`pacote_exemplo/modulo.py`**
```python
def diga_ola():
    print("Olá do módulo!")
```

**`pacote_exemplo/__init__.py`**
```python
from .modulo import diga_ola  # reexporta
```

**Uso**
```python
from pacote_exemplo import diga_ola
diga_ola()
```



---

### Arquivos gerados neste notebook
- `data/vendas.csv`, `data/clientes.csv`, `data/filmes.json`, `data/dados.xlsx`
- `output/amostra.csv`, `output/amostra.xlsx`, `output/amostra.json`, `output/amostra.parquet` (se suportado)
- `output/tabela_mensal.xlsx`
